# **Text Classification: Sparse Transfer Learning with the CLI**

In this example, you will fine-tune a 90% pruned BERT model onto some text classification datasets using SparseML's CLI.

### **Sparse Transfer Learning Overview**

Sparse Transfer Learning is very similiar to typical fine-tuning you are used to when training models. However, with Sparse Transfer Learning, we start the training process from a pre-sparsified checkpoint and maintain the sparsity structure while the fine tuning occurs. At the end, you will have a sparse model trained on your dataset, ready to be deployed with DeepSparse for GPU-class performance on CPUs!

### **Pre-Sparsified BERT**
SparseZoo, Neural Magic's open source repository of pre-sparsified models, contains a 90% pruned version of BERT, which has been sparsified on the upstream Wikipedia and BookCorpus datasets with the
masked language modeling objective. [Check out the model card](https://sparsezoo.neuralmagic.com/models/nlp%2Fmasked_language_modeling%2Fobert-base%2Fpytorch%2Fhuggingface%2Fwikipedia_bookcorpus%2Fpruned90-none). We will use this model as the starting point for the transfer learning process.


***Let's dive in!***

## **Installation**

Install SparseML via `pip`.



In [1]:
%pip uninstall torch torchvision -y
%pip install sparseml[torch]

Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116
Found existing installation: torchvision 0.14.1+cu116
Uninstalling torchvision-0.14.1+cu116:
  Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.8/904.8 KB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

If you are running on Google Colab, restart the runtime after this step.

In [1]:
!sparseml.transformers.text_classification --help

2023-02-27 03:35:42 sparseml.transformers WARNING  sparseml-transformers v4.23.1 installation not detected. Installing  sparseml-transformers v4.23.1 dependencies if transformers is already  installed in the environment, it will be overwritten. Set  environment variable NM_NO_AUTOINSTALL_TRANSFORMERS to disable
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 KB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

## **Sparse Transfer Learning with MNLI**

SparseML's CLI enables you to kick-off training workflows with various utilities like dataset loading, checkpoint saving, 
metric reporting, and logging handled for you. All we have to do is pass a `model_name_or_path` (the starting checkpoint), a `task` (the GLUE task to train on), and a `recipe` (a YAML file specifying the sparsity related parameters) and we are up and running. The `recipes` is critical for instructing the training script how to modify the training process with sparsity related algorithms. For Sparse Transfer Learning, we will use a `recipe` that instructs SparseML to maintain sparsity during the training process and to apply quantization over the final few epochs. 

### **Run Transfer Learning**

For MNLI, there is a pre-made transfer learning recipe available in [SparseZoo](https://sparsezoo.neuralmagic.com/models/nlp%2Ftext_classification%2Fobert-base%2Fpytorch%2Fhuggingface%2Fmnli%2Fpruned90_quant-none). As such, we kick off transfer learning with the following:

In [ ]:
!sparseml.transformers.train.text_classification \
  --task_name mnli \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none \
  --distill_teacher zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/base-none \
  --output_dir sparse_quantized_bert-text_classification_mnli \
  --do_train --do_eval --max_seq_length 128 --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 8 --per_device_eval_batch_size 32 --gradient_accumulation_steps 4 --preprocessing_num_workers 32 \
  --seed 5114

Let's discuss the key arguments:
- `--task_name mnli` instructs SparseML to download and fine-tune onto the MNLI dataset. You can pass any GLUE task as the task name and SparseML automatically downloads the dataset from the Hugging Face hub.

- `--model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none` specifies the starting checkpoint for the fine tuning. Here, we passed a SparseZoo stub identifying the 90% pruned version of BERT trained with masked language modeling on the Wikipedia and BookCorpus datasets. SparseML downloads the checkpoint from the Zoo when the script starts.

- `--recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none` specifies the recipe to be applied by SparseML. Here, we passed a SparseZoo stub identifying the transfer learning recipe for the MNLI dataset. SparseML downloads the recipe from the Zoo when the script starts. See below for the details of what this recipe looks like.

- `--distill_teacher zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/base-none` is an optional argument that specifies a model to use for as a teacher to apply distillation during the training process. We passed a SparseZoo stub identifying a dense BERT model trained on MNLI. SparseML downloads the teacher from the Zoo when the script starts.

The script downloads the starting checkpoint, the teacher model, and transfer learning recipe from SparseZoo as well as the MNLI 
dataset and trains the model for 12 epochs, converging to ~82.5% accuracy on the validation set.

#### **Transfer Learning Recipe**

Here's what the transfer learning recipe for the MNLI dataset looks like.

The "Modifiers" are the important items that encode how SparseML should modify the training process for Sparse Transfer Learning:
- `ConstantPruningModifier` tells SparseML to pin weights at 0 over all epochs, maintaining the sparsity structure of the network
- `QuantizationModifier` tells SparseML to quanitze the weights with quantization aware training over the last 5 epochs
- `DistillationModifier` tells SparseML how to apply distillation to the model, including the layer and some hyperparameters

SparseML parses the modifiers and updates the training process to implement the algorithms and hyperparameters specified in the recipes.

In [10]:
from sparsezoo import Model
transfer_stub = "zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none"
download_dir = "./transfer_recipe-mnli"
zoo_model = Model(transfer_stub, download_path=download_dir)
recipe_path = zoo_model.recipes.default.path
print(recipe_path)

downloading...:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

./transfer_recipe-mnli/recipe/recipe_original.md


In [11]:
%cat ./transfer_recipe-mnli/recipe/recipe_original.md

<!--
Copyright (c) 2021 - present / Neuralmagic, Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
-->

---

version: 1.1.0

num_epochs: 13
init_lr: 8e-5
final_lr: 0

qat_start_epoch: 8.0
observer_epoch: 12.0
quantize_embeddings: 1

distill_hardness: &distill_hardness 1.0
distill_temperature: &distill_temperature 3.0

weight_decay: 0.0

# Modifiers:

training_modifiers:
  - !EpochRangeModifier
      end_epoch: eval(num_epochs)
      start_epoch: 0.0

  - !LearningRateFunctionModifier
      start

### **Export to ONNX**

Once you have trained your model, export to ONNX in order to deploy with DeepSparse. The artifacts of the training process 
are saved to your local filesystem. 

Run the following to convert your PyTorch checkpoint to ONNX:

In [ ]:
!sparseml.transformers.export_onnx \
  --model_path sparse_quantized_bert-text_classification_mnli \
  --task text_classification

The script above creates a `deployment` folder in your local directory, which has all of the files needed for deployment with DeepSparse including the `model.onnx`, `config.json`, and `tokenizer.json` files.

## **Sparse Transfer Learning with QQP**

SparseML's CLI enables you to kick-off training workflows with various utilities like dataset loading, checkpoint saving, 
metric reporting, and logging handled for you. All we have to do is pass a `model_name_or_path` (the starting checkpoint), a `task` (the GLUE task to train on), and a `recipe` (a YAML file specifying the sparsity related parameters) and we are up and running. The `recipes` is critical for instructing the training script how to modify the training process with sparsity related algorithms. For Sparse Transfer Learning, we will use a `recipe` that instructs SparseML to maintain sparsity during the training process and to apply quantization over the final few epochs. 

### **Run Transfer Learning**

For QQP, there is a pre-made transfer learning recipe available in [SparseZoo](https://sparsezoo.neuralmagic.com/models/nlp%2Ftext_classification%2Fobert-base%2Fpytorch%2Fhuggingface%2Fqqp%2Fpruned90_quant-none). As such, we kick off transfer learning with the following:

In [ ]:
!sparseml.transformers.train.text_classification \
  --task_name qqp \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/qqp/pruned90_quant-none \
  --distill_teacher zoo:nlp/text_classification/obert-base/pytorch/huggingface/qqp/base-none \
  --output_dir obert_base_pruned90_quant_qqp \
  --do_train --do_eval --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 32 --per_device_eval_batch_size 32 --preprocessing_num_workers 32 \
  --max_seq_length 128 \
  --seed 10194

Let's discuss the key arguments:
- `--task_name qqp` instructs SparseML to download and fine-tune onto the QQP dataset. You can pass any GLUE task as the task name and SparseML automatically downloads the dataset from the Hugging Face hub.

- `--model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none` specifies the starting checkpoint for the fine tuning. Here, we passed a SparseZoo stub identifying the 90% pruned version of BERT trained with masked language modeling on the Wikipedia and BookCorpus datasets. SparseML downloads the checkpoint from the Zoo when the script starts.

- `--recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/qqp/pruned90_quant-none` specifies the recipe to be applied by SparseML. Here, we passed a SparseZoo stub identifying the transfer learning recipe for the QQP dataset. SparseML downloads the recipe from the Zoo when the script starts. See below for the details of what this recipe looks like.

- `--distill_teacher zoo:nlp/text_classification/obert-base/pytorch/huggingface/qqp/base-none` is an optional argument that specifies a model to use for as a teacher to apply distillation during the training process. We passed a SparseZoo stub identifying a dense BERT model trained on QQP. SparseML downloads the teacher from the Zoo when the script starts.

The script downloads the starting checkpoint, the teacher model, and transfer learning recipe from SparseZoo as well as the QQP 
dataset and trains the model for 3 epochs, converging to ~91% accuracy on the validation set.

#### **Transfer Learning Recipe**

Here's what the transfer learning recipe for the QQP dataset looks like.

The "Modifiers" are the important items that encode how SparseML should modify the training process for Sparse Transfer Learning:
- `ConstantPruningModifier` tells SparseML to pin weights at 0 over all epochs, maintaining the sparsity structure of the network
- `QuantizationModifier` tells SparseML to quanitze the weights with quantization aware training over the last 5 epochs
- `DistillationModifier` tells SparseML how to apply distillation to the model, including the layer and some hyperparameters

SparseML parses the modifiers and updates the training process to implement the algorithms and hyperparameters specified in the recipes.

In [8]:
from sparsezoo import Model
transfer_stub = "zoo:nlp/text_classification/obert-base/pytorch/huggingface/qqp/pruned90_quant-none"
download_dir = "./transfer_recipe-qqp"
zoo_model = Model(transfer_stub, download_path=download_dir)
recipe_path = zoo_model.recipes.default.path
print(recipe_path)

downloading...:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

./transfer_recipe-qqp/recipe/recipe_original.md


In [9]:
%cat ./transfer_recipe-qqp/recipe/recipe_original.md

<!--
Copyright (c) 2021 - present / Neuralmagic, Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
-->

---

version: 1.1.0

# General Variables
num_epochs: &num_epochs 13
init_lr: 1.5e-4 
final_lr: 0

qat_start_epoch: &qat_start_epoch 8.0
observer_epoch: &observer_epoch 12.0
quantize_embeddings: &quantize_embeddings 1

distill_hardness: &distill_hardness 1.0
distill_temperature: &distill_temperature 2.0

# Modifiers:

training_modifiers:
  - !EpochRangeModifier
      end_epoch: eval(num_epochs)

### **Export to ONNX**

Once you have trained your model, export to ONNX in order to deploy with DeepSparse. The artifacts of the training process 
are saved to your local filesystem. 

Run the following to convert your PyTorch checkpoint to ONNX:

In [ ]:
!sparseml.transformers.export_onnx \
  --model_path obert_base_pruned90_quant_qqp \
  --task text_classification

The script above creates a `deployment` folder in your local directory, which has all of the files needed for deployment with DeepSparse including the `model.onnx`, `config.json`, and `tokenizer.json` files.

## **Sparse Transfer Learning with a Custom Dataset: Single Input Classification**

Beyond the built-in GLUE tasks, we can also use datasets for single input multi-class classification problems. The datasets can either be passed as Hugging Face hub model identifiers or via local CSV/JSON files.

Let's try to transfer onto the [TweetEval Emotion Dataset](https://huggingface.co/datasets/tweet_eval). This dataset 
contains single sentences with 4 labels representing the emotion of the tweet (`0=anger, 1=joy, 2=optimism, 3=sadness`.

For simplicity, we will perform the fine-tuning without distillation. Although the transfer learning recipe contains distillation modifiers, we can turn them off by setting `--distill_teacher disable`.

### **Using a Hugging Face Dataset**

Let's walk through how to pass a Hugging Face dataset identifier to the CLI.

#### **Inspecting TweetEval Dataset**

In [12]:
from datasets import load_dataset
from pprint import pprint

emotion = load_dataset("tweet_eval", "emotion")
print(emotion)
pprint(emotion["train"][0])

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/134k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 374
    })
})
{'label': 2,
 'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce "
         'Meyer.  #motivation #leadership #worry'}


#### **Kick off Training**

To configure the training script to use the emotion dataset, we replaced the `--task_name` argument with `--dataset_name tweet_eval --dataset_config_name emotion --input_column_names text --label_column_name label`. Since the TweetEval dataset contains multiple subsets (e.g. there is a subset
that classifies text into an emoji), we pass the `--dataset_config_name` to specify the subset.


Run the following to kick of the training process:

In [ ]:
!sparseml.transformers.train.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none \
  --recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}' \
  --distill_teacher disable \
  --dataset_name tweet_eval --dataset_config_name emotion \
  --input_column_names "text" --label_column_name "label" \
  --output_dir sparse_quantized_bert-text_classification_tweet_eval_emotion \
  --do_train --do_eval --max_seq_length 128 --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 8 --per_device_eval_batch_size 32 --gradient_accumulation_steps 4 --preprocessing_num_workers 32 \
  --seed 5114

You will notice that we used the same recipe as we did in the SST2 case (identified by the SparseZoo stub `zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none`). 

Since the TweetEval Emotion dataset is a single sentence multi-class classification problem, we used the transfer learning recipe from the sentiment analysis task (a single sentence binary classification problem) as the starting point.



To update a recipe, you can download the YAML file from SparseZoo, make updates to the YAML directly, and pass the local path to SparseML. In this case, we used `--recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}'` to modify a recipe on the fly, updating to only run for 11 epochs.

### **Using Local CSV/JSON Files**

Let's walk through how to pass a CSV/JSON dataset to the CLI.

#### **Save Dataset as a CSV File**

For this example, we use Hugging Face `datasets` to create a CSV file for Tweet Eval dataset that can be passed to SparseML's CLI but you can use any framework you want to create the CSV.

In [15]:
from datasets import load_dataset
from pprint import pprint

emotion = load_dataset("tweet_eval", "emotion")
print(emotion)
pprint(emotion["train"][0])

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3257
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1421
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 374
    })
})
{'label': 2,
 'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce "
         'Meyer.  #motivation #leadership #worry'}


In [16]:
emotion["train"].to_csv("./emotion-train.csv")
emotion["validation"].to_csv("./emotion-validation.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

36463

We can see that the data is a CSV file with text and label as the columns:

In [17]:
!head ./emotion-train.csv --lines=10

,text,label
0,“Worry is a down payment on a problem you may never have'.  Joyce Meyer.  #motivation #leadership #worry,2
1,My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs,0
2,No but that's so cute. Atsu was probably shy about photos before but cherry helped her out uwu,1
3,"Rooneys fucking untouchable isn't he? Been fucking dreadful again, depay has looked decent(ish)tonight",0
4,it's pretty depressing when u hit pan on ur favourite highlighter,3
5,@user but your pussy was weak from what I heard so stfu up to me bitch . You got to threaten him that your pregnant .,0
6,Making that yearly transition from excited and hopeful college returner to sick and exhausted pessimist. #college,3
7,Tiller and breezy should do a collab album. Rapping and singing prolly be fire,1
8,@user broadband is shocking regretting signing up now #angry #shouldofgonewithvirgin,0


In [18]:
!head ./emotion-validation.csv --lines=10

,text,label
0,"@user @user Oh, hidden revenge and anger...I rememberthe time,she rebutted you.",0
1,if not then #teamchristine bc all tana has done is provoke her by tweeting shady shit and trying to be a hard bitch begging for a fight,0
2,Hey @user #Fields in #skibbereen give your online delivery service a horrible name. 1.5 hours late on the 1 hour delivery window.,0
3,Why have #Emmerdale had to rob #robron of having their first child together for that vile woman/cheating sl smh #bitter,0
4,@user I would like to hear a podcast of you going off refuting her entire article. Extra indignation please.,0
5,If I have to hear one more time how I am intimidate men... I'm going to explode! Why are guys these days so pussified?,0
6,depression sucks😔,3
7,"O, the melancholy Catacombs quickly wandered about the Rue Morgue, Madman!",3
8,#RIPBiwott I think Robert oukos soul can now rejoice and rest in peace. Call an evil man evil and a good man a good man. He was an evil man,0


#### **Kick off Training**

To use the local files with the CLI, pass `--train_file ./emotion-train.csv --validation_file ./emotion-validation.csv  --input_column_names text --label_column_name label`.

Run the following:

In [ ]:
!sparseml.transformers.train.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none \
  --recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}' \
  --distill_teacher disable \
  --train_file ./emotion-train.csv --validation_file ./emotion-validation.csv \
  --input_column_names "text" --label_column_name "label" \
  --output_dir sparse_quantized_bert-text_classification_tweet_eval_emotion-csv \
  --do_train --do_eval --max_seq_length 128 --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 8 --per_device_eval_batch_size 32 --gradient_accumulation_steps 4 --preprocessing_num_workers 32 \
  --seed 5114

## **Sparse Transfer Learning with a Custom Dataset: Multiple Input Classification**

Beyond the built-in GLUE tasks, we can also use datasets for multiple input multi-class classification problems. The datasets can either be passed as Hugging Face hub model identifiers or via local CSV/JSON files.

Let's try to transfer onto the [SICK (Sentences Involving Compositional Knowldedge) Dataset](https://huggingface.co/datasets/sick),
which includes 10,000 pairs of sentences with entailment relationships (`0=entailment, 1=neural, 2=contradiction`).

For simplicity, we will perform the fine-tuning without distillation. Although the transfer learning recipe contains distillation modifiers, we can turn them off by setting `--distill_teacher disable`.

### **Using a Hugging Face Dataset**

Let's walk through how to pass a Hugging Face dataset identifier to the CLI.

#### **Inspecting SICK Dataset**

`Sentence_A` and `Sentence_B` are the input columns. `label` is the label column.

In [20]:
from datasets import load_dataset
from pprint import pprint

sick = load_dataset("sick")
print(sick)
pprint(sick["train"][0])

Downloading:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/218k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset sick downloaded and prepared to /root/.cache/huggingface/datasets/sick/default/0.0.0/c6b3b0b44eb84b134851396d6d464e5cb8f026960519d640e087fe33472626db. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4906
    })
})
{'entailment_AB': 'A_neutral_B',
 'entailment_BA': 'B_neutral_A',
 'id': '1',
 'label': 1,
 'relatedness_score': 4.5,
 'sentence_A': 'A group of kids is playing in a yard and an old man is '
            

#### **Kick off Training**

To configure the training script to use the emotion dataset, we replaced the `--task_name` argument with `--dataset_name sick --input_column_names 'sentence_A,sentence_B' --label_column_name label`.


Run the following to kick of the training process:

In [ ]:
!sparseml.transformers.train.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none \
  --distill_teacher disable \
  --dataset_name sick --input_column_names 'sentence_A,sentence_B' --label_column_name 'label' \
  --output_dir sparse_quantized_bert-text_classification_sick \
  --do_train --do_eval --max_seq_length 128 --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 8 --per_device_eval_batch_size 32 --gradient_accumulation_steps 4 --preprocessing_num_workers 32 \
  --seed 5114

You will notice that we used the same recipe as we did in the MNLI case (identified by the SparseZoo stub `zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none`). 

Since the MNLI dataset is a multi sentence multi-class classification problem (similiarly, it is an entailment problem), we used the transfer learning recipe from the sentiment analysis task (a single sentence binary classification problem) as the starting point.

To update a recipe, you can download the YAML file from SparseZoo, make updates to the YAML directly, and pass the local path to SparseML. Alternative, you can use `--recipe_args` to update on the fly.

### **Using Local CSV/JSON Files**

Let's walk through how to pass a CSV/JSON dataset to the CLI.

#### **Save Dataset as a CSV File**

For this example, we use Hugging Face `datasets` to create a CSV file for Tweet Eval dataset that can be passed to SparseML's CLI but you can use any framework you want to create the CSV.

In [21]:
from datasets import load_dataset
from pprint import pprint

sick = load_dataset("sick")
print(sick)
pprint(sick["train"][0])

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4439
    })
    validation: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 495
    })
    test: Dataset({
        features: ['id', 'sentence_A', 'sentence_B', 'label', 'relatedness_score', 'entailment_AB', 'entailment_BA', 'sentence_A_original', 'sentence_B_original', 'sentence_A_dataset', 'sentence_B_dataset'],
        num_rows: 4906
    })
})
{'entailment_AB': 'A_neutral_B',
 'entailment_BA': 'B_neutral_A',
 'id': '1',
 'label': 1,
 'relatedness_score': 4.5,
 'sentence_A': 'A group of kids is playing in a yard and an old man is '
            

In [22]:
sick["train"].to_csv("./sick-train.csv")
sick["validation"].to_csv("./sick-validation.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

118755

We can see that the data is a CSV file with `sentence_A` and `sentence_B` as the input columns and `label` as the label column:

In [25]:
!head ./sick-train.csv --lines=10

,id,sentence_A,sentence_B,label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset
0,1,A group of kids is playing in a yard and an old man is standing in the background,A group of boys in a yard is playing and a man is standing in the background,1,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man in the background.","A group of children playing in a yard, a man in the background.",FLICKR,FLICKR
1,2,A group of children is playing in the house and there is no man standing in the background,A group of kids is playing in a yard and an old man is standing in the background,1,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man in the background.","A group of children playing in a yard, a man in the background.",FLICKR,FLICKR
2,3,The young boys are playing outdoors and the man is smiling nearby,The kids are playing outdoors near a man with a smile,0,4.7,A_entails_B,B_enta

In [26]:
!head ./sick-validation.csv --lines=10

,id,sentence_A,sentence_B,label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset
0,4,The young boys are playing outdoors and the man is smiling nearby,There is no boy playing outdoors and there is no man smiling,2,3.6,A_contradicts_B,B_contradicts_A,"The children are playing outdoors, while a man smiles nearby.","The children are playing outdoors, while a man smiles nearby.",FLICKR,FLICKR
1,24,A person in a black jacket is doing tricks on a motorbike,A skilled person is riding a bicycle on one wheel,1,3.4,A_neutral_B,B_neutral_A,A person in a black jacket doing tricks on a motorbike.,A person rides their bicycle onto a rock and balances on one wheel.,FLICKR,FLICKR
2,105,Four children are doing backbends in the gym,Four girls are doing backbends and playing outdoors,1,3.8,A_neutral_B,B_neutral_A,Four girls do backbends while playing outdoors.,Four children do backbends in the park.,FLICKR,FLICKR
3,116,A player is

#### **Kick off Training**

To use the local files with the CLI, pass `--train_file ./sick-train.csv --validation_file ./sick-validation.csv  --input_column_names 'sentence_A,sentence_B' --label_column_name label`.

Run the following:

In [27]:
!sparseml.transformers.train.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/text_classification/obert-base/pytorch/huggingface/mnli/pruned90_quant-none \
  --distill_teacher disable \
  --train_file ./sick-train.csv --validation_file ./sick-validation.csv --input_column_names 'sentence_A,sentence_B' --label_column_name 'label' \
  --output_dir sparse_quantized_bert-text_classification_sick-csv \
  --do_train --do_eval --max_seq_length 128 --evaluation_strategy epoch --logging_steps 1000 --save_steps 1000 \
  --per_device_train_batch_size 8 --per_device_eval_batch_size 32 --gradient_accumulation_steps 4 --preprocessing_num_workers 32 \
  --seed 5114

2023-02-27 04:05:30.285502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 04:05:31.867343: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 04:05:31.867515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 04:05:31.867542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War